# Pipeline tiền xử lý dữ liệu dạng bảng (Notebook nộp)

**Dữ liệu:** fraudTest.csv — phát hiện giao dịch gian lận.  
**Cấu trúc notebook:**
1. Nạp dữ liệu & xử lý ban đầu  
2. 2a — Xử lý giá trị khuyết (Missing Values)  
3. 2b — Chuẩn hoá dữ liệu (Normalization)  
4. 2c — Mã hoá biến phân loại (Encoding)  
5. 2d — Lựa chọn đặc trưng (Feature Selection)  
6. Lưu dữ liệu đầu ra

**Phương án cuối cùng đã chọn (chi tiết so sánh có trong report):**
- 2a: Dữ liệu không thiếu, fallback = median/mode nếu cần  
- 2b: Chuẩn hoá bằng **Z-score Standardization** và **Robust**
- 2c: **One-Hot** cho cột ≤60 mức (gộp nhãn hiếm 0.5%), **Frequency Encoding** cho cột nhiều mức  
- 2d: **VarianceThreshold** để loại cột hằng  
- Loại các cột định danh (ID/PII) không hữu ích


## Import libs

In [1]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold

## Exploring data

In [2]:
# Hiển thị dữ liệu (Chưa xử lý)

data_path = Path("../data/tabular/fraudTest.csv")
df_raw = pd.read_csv(data_path)

print("TỔNG QUAN DỮ LIỆU BAN ĐẦU")
print("────────────────────────────")
print(f"Số hàng (rows): {df_raw.shape[0]:,}")
print(f"Số cột  (columns): {df_raw.shape[1]}")
print("\nDanh sách tên cột:")
print(df_raw.columns.tolist())

print("\nKiểu dữ liệu của các cột:")
display(df_raw.dtypes.to_frame("Data type").T)

# Kiểm tra giá trị thiếu
missing = df_raw.isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print("\nCác cột có giá trị bị thiếu:")
    display(missing.to_frame("Số lượng thiếu"))
else:
    print("\nKhông phát hiện giá trị bị thiếu trong tập dữ liệu.")

# Thống kê cơ bản các cột số
print("\nThống kê cơ bản các cột số:")
display(df_raw.describe().T)

# Thống kê nhanh các cột phân loại
cat_cols = df_raw.select_dtypes(include=["object"]).columns.tolist()
if cat_cols:
    print(f"\nTổng số cột phân loại: {len(cat_cols)}")
    for c in cat_cols[:10]:
        print(f"  - {c:<25} → {df_raw[c].nunique():>5} giá trị duy nhất")
    if len(cat_cols) > 10:
        print(f"  ... (và {len(cat_cols) - 10} cột phân loại khác)")
else:
    print("\nKhông có cột dạng object (phân loại).")

# Hiển thị 5 dòng đầu tiên
print("\n5 dòng dữ liệu đầu tiên:")
display(df_raw.head())

# Nếu muốn xem 5 dòng ngẫu nhiên:
# display(df_raw.sample(5, random_state=42))


TỔNG QUAN DỮ LIỆU BAN ĐẦU
────────────────────────────
Số hàng (rows): 555,719
Số cột  (columns): 23

Danh sách tên cột:
['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']

Kiểu dữ liệu của các cột:


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
Data type,int64,object,int64,object,object,float64,object,object,object,object,...,float64,float64,int64,object,object,object,int64,float64,float64,int64



Không phát hiện giá trị bị thiếu trong tập dữ liệu.

Thống kê cơ bản các cột số:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,555719.0,2.778590e+05,1.604224e+05,0.000000e+00,1.389295e+05,2.778590e+05,4.167885e+05,5.557180e+05
cc_num,555719.0,4.178387e+17,1.309837e+18,6.041621e+10,1.800429e+14,3.521417e+15,4.635331e+15,4.992346e+18
amt,555719.0,6.939281e+01,1.567459e+02,1.000000e+00,9.630000e+00,4.729000e+01,8.301000e+01,2.276811e+04
zip,555719.0,4.884263e+04,2.685528e+04,1.257000e+03,2.629200e+04,4.817400e+04,7.201100e+04,9.992100e+04
lat,555719.0,3.854325e+01,5.061336e+00,2.002710e+01,3.466890e+01,3.937160e+01,4.189480e+01,6.568990e+01
long,555719.0,-9.023131e+01,1.372178e+01,-1.656723e+02,-9.679800e+01,-8.747690e+01,-8.017520e+01,-6.795030e+01
city_pop,555719.0,8.822189e+04,3.003909e+05,2.300000e+01,7.410000e+02,2.408000e+03,1.968500e+04,2.906700e+06
unix_time,555719.0,1.380679e+09,5.201104e+06,1.371817e+09,1.376029e+09,1.380762e+09,1.385867e+09,1.388534e+09
merch_lat,555719.0,3.854280e+01,5.095829e+00,1.902742e+01,3.475530e+01,3.937659e+01,4.195416e+01,6.667930e+01
merch_long,555719.0,-9.023138e+01,1.373307e+01,-1.666716e+02,-9.690513e+01,-8.744520e+01,-8.026464e+01,-6.695203e+01



Tổng số cột phân loại: 12
  - trans_date_trans_time     → 544760 giá trị duy nhất
  - merchant                  →   693 giá trị duy nhất
  - category                  →    14 giá trị duy nhất
  - first                     →   341 giá trị duy nhất
  - last                      →   471 giá trị duy nhất
  - gender                    →     2 giá trị duy nhất
  - street                    →   924 giá trị duy nhất
  - city                      →   849 giá trị duy nhất
  - state                     →    50 giá trị duy nhất
  - job                       →   478 giá trị duy nhất
  ... (và 2 cột phân loại khác)

5 dòng dữ liệu đầu tiên:


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [3]:
# xử lý nhẹ dữ liệu 

df = df_raw.copy()

# 1) Xoá cột index thừa
if "Unnamed: 0" in df.columns:
    df.drop(columns=["Unnamed: 0"], inplace=True)
    print("Đã xoá cột 'Unnamed: 0' (index thừa).")

# 2) Chuyển kiểu ngày–giờ cho cột thời gian
if "trans_date_trans_time" in df.columns:
    df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"], errors="coerce")
    print("Đã chuyển 'trans_date_trans_time' sang kiểu datetime.")

# 3) Xác định cột nhãn mục tiêu
TARGET = "is_fraud"
assert TARGET in df.columns, "Không tìm thấy cột nhãn 'is_fraud' trong dữ liệu."

# 4) In thông tin sau xử lý sơ bộ
print("\nKích thước sau xử lý sơ bộ:", df.shape)
print("Các cột hiện có:", df.columns.tolist())

# 5) Xem nhanh 3 dòng đầu sau xử lý
display(df.head(3))


Đã xoá cột 'Unnamed: 0' (index thừa).
Đã chuyển 'trans_date_trans_time' sang kiểu datetime.

Kích thước sau xử lý sơ bộ: (555719, 22)
Các cột hiện có: ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0


## 2a — Xử lý giá trị khuyết (Missing Values)

**Mục tiêu:**  
Kiểm tra dữ liệu để xác định các ô bị thiếu, sau đó xử lý hợp lý mà không làm thay đổi dữ liệu sạch.

**Phương pháp:**  
- Bước 1: Kiểm tra toàn bộ tập dữ liệu để xác định các ô NaN (thiếu giá trị).  
- Bước 2: Nếu không có dữ liệu thiếu → giữ nguyên.  
- Bước 3: Nếu có →  
  - Với biến **số học**: điền bằng **median** (giá trị trung vị).  
  - Với biến **phân loại**: điền bằng **mode** (giá trị xuất hiện nhiều nhất).  
- Bước 4: Hiển thị rõ **giá trị được điền vào từng ô bị thiếu** để đảm bảo minh bạch.


In [4]:
# 2a — Xử lý giá trị khuyết

# Kiểm tra tình trạng dữ liệu bị thiếu
missing_info = df.isna().sum()
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)

if len(missing_info) == 0:
    print("Không phát hiện giá trị bị thiếu trong tập dữ liệu.")
else:
    print("Các cột có giá trị bị thiếu:")
    display(missing_info.to_frame("Số lượng thiếu"))

    # Lưu vị trí các ô bị thiếu
    missing_positions = np.argwhere(df.isna().values)

    print("\nVị trí các ô bị thiếu (hàng, cột):")
    for row_idx, col_idx in missing_positions:
        col_name = df.columns[col_idx]
        print(f"  - Hàng {row_idx}, cột '{col_name}' → NaN")

    # Tiến hành xử lý (median cho số, mode cho phân loại)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

    print("\nTiến hành xử lý giá trị khuyết:")

    for c in num_cols:
        if df[c].isna().any():
            median_val = df[c].median()
            df[c] = df[c].fillna(median_val)
            print(f"  - Cột '{c}': điền {median_val:.3f} (Median).")

    for c in cat_cols:
        if df[c].isna().any():
            mode_val = df[c].mode()[0]
            df[c] = df[c].fillna(mode_val)
            print(f"  - Cột '{c}': điền '{mode_val}' (Mode).")

    # In ra các ô đã được điền lại
    print("\nSau khi xử lý, các ô bị thiếu đã được cập nhật như sau:")
    for row_idx, col_idx in missing_positions:
        col_name = df.columns[col_idx]
        filled_val = df.iloc[row_idx, col_idx]
        print(f"  - Hàng {row_idx}, cột '{col_name}' → {filled_val}")

    # Bước 5. Kiểm tra lại
    total_missing_after = df.isna().sum().sum()
    print(f"\nTổng số ô còn thiếu sau xử lý: {total_missing_after}")


Không phát hiện giá trị bị thiếu trong tập dữ liệu.


## 2b — Chuẩn hoá dữ liệu (Normalization)

**Mục tiêu:**  
Đưa các đặc trưng số về cùng thang đo nhằm giúp mô hình học máy hội tụ nhanh hơn và không bị lệch do đơn vị đo khác nhau.

**Phương pháp:**  
Áp dụng **Z-score Standardization** cho các đặc trưng số liên tục theo công thức:

$$
z = \frac{x - \mu}{\sigma}
$$

Trong đó:  
- $\mu$: giá trị trung bình của cột  
- $\sigma$: độ lệch chuẩn của cột  

**Robust Scaling (Median–IQR)** 
Chuẩn hoá theo **trung vị** và **độ trải IQR** để giảm ảnh hưởng của ngoại lai/đuôi dài.

**Công thức:**  

$$
x'=\frac{x-\mathrm{median}(x)}{\mathrm{IQR}(x)},\quad \mathrm{IQR}=Q_3-Q_1
$$

**Dùng khi:**  
- Biến có **phân phối lệch mạnh/đuôi dài** (vd: `amt`).  
- Tồn tại **nhiều outlier** khiến ($\mu$,$\sigma$) của Z-score kém ổn định.

In [5]:

TARGET = "is_fraud"

# Chọn cột số (trừ TARGET) và bỏ cột nhị phân/ID nếu muốn
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != TARGET]

binary_like = [c for c in num_cols if df[c].nunique(dropna=True) == 2]
id_or_discrete = [c for c in ["cc_num", "zip", "trans_num"] if c in df.columns]
num_cols_clean = [c for c in num_cols if c not in binary_like + id_or_discrete]

# Chia nhóm Robust/Z theo skew (không sửa df)
robust_cols = ["amt"] if "amt" in num_cols_clean else []
skews = df[num_cols_clean].skew(numeric_only=True)
zscore_cols = [c for c in num_cols_clean if c not in robust_cols]
for c in zscore_cols[:]:
    if abs(skews.get(c, 0)) > 1.0:
        robust_cols.append(c); zscore_cols.remove(c)

# Scale ra mảng float, rồi dựng DataFrame MỚI
scaled_parts = []
if zscore_cols:
    zs = StandardScaler().fit_transform(df[zscore_cols].to_numpy(dtype=float))
    scaled_parts.append(pd.DataFrame(zs, columns=[f"zs__{c}" for c in zscore_cols], index=df.index))
if robust_cols:
    rb = RobustScaler(quantile_range=(25, 75)).fit_transform(df[robust_cols].to_numpy(dtype=float))
    scaled_parts.append(pd.DataFrame(rb, columns=[f"rob__{c}" for c in robust_cols], index=df.index))

df_scaled = pd.concat(scaled_parts, axis=1) if scaled_parts else pd.DataFrame(index=df.index)

# So sánh nhanh
print("Thống kê gốc (một phần):")
display(df[num_cols_clean].describe().T)

print("\nThống kê sau chuẩn hoá (DataFrame mới, không sửa df):")
display(df_scaled.describe().T)


Thống kê gốc (một phần):


,count,mean,std,min,25%,50%,75%,max
amt,555719.0,6.939281e+01,1.567459e+02,1.000000e+00,9.630000e+00,4.729000e+01,8.301000e+01,2.276811e+04
lat,555719.0,3.854325e+01,5.061336e+00,2.002710e+01,3.466890e+01,3.937160e+01,4.189480e+01,6.568990e+01
long,555719.0,-9.023131e+01,1.372178e+01,-1.656723e+02,-9.679800e+01,-8.747690e+01,-8.017520e+01,-6.795030e+01
city_pop,555719.0,8.822189e+04,3.003909e+05,2.300000e+01,7.410000e+02,2.408000e+03,1.968500e+04,2.906700e+06
unix_time,555719.0,1.380679e+09,5.201104e+06,1.371817e+09,1.376029e+09,1.380762e+09,1.385867e+09,1.388534e+09
merch_lat,555719.0,3.854280e+01,5.095829e+00,1.902742e+01,3.475530e+01,3.937659e+01,4.195416e+01,6.667930e+01
merch_long,555719.0,-9.023138e+01,1.373307e+01,-1.666716e+02,-9.690513e+01,-8.744520e+01,-8.026464e+01,-6.695203e+01



Thống kê sau chuẩn hoá (DataFrame mới, không sửa df):


,count,mean,std,min,25%,50%,75%,max
zs__lat,555719.0,-1.244232e-15,1.000001,-3.658356,-0.765481,0.163662,0.662187,5.363538
zs__unix_time,555719.0,1.904686e-14,1.000001,-1.703871,-0.894087,0.015982,0.997504,1.510355
zs__merch_lat,555719.0,-3.089803e-15,1.000001,-3.829680,-0.743255,0.163623,0.669443,5.521481
rob__amt,555719.0,3.012103e-01,2.136085,-0.630826,-0.513219,0.000000,0.486781,309.632325
rob__long,555719.0,-1.657010e-01,0.825480,-4.704105,-0.560742,0.000000,0.439258,1.174688
rob__city_pop,555719.0,4.529872e+00,15.856783,-0.125897,-0.087996,0.000000,0.912004,153.309333
rob__merch_long,555719.0,-1.674335e-01,0.825280,-4.761059,-0.568488,0.000000,0.431512,1.231525


## 2c — Mã hoá biến phân loại (Categorical Encoding)

**Mục tiêu:**  
Chuyển các cột dạng chữ (object/categorical) thành số để mô hình học máy sử dụng được.

**Phương án đã chọn:**
- **One-Hot Encoding** cho cột có **≤ 60** giá trị khác nhau (có **gộp nhãn hiếm < 0.5%** thành `__OTHER__`).
- **Frequency Encoding** cho cột **> 60** giá trị (high-cardinality), ánh xạ mỗi nhãn → tần suất xuất hiện trong cột.

In [6]:
TARGET = "is_fraud"
assert TARGET in df.columns, "Thiếu cột nhãn is_fraud."

X_full = df.drop(columns=[TARGET]).copy()
y_full = df[TARGET].astype(int).copy()

# Xác định cột phân loại
cat_cols = X_full.select_dtypes(include=["object", "category"]).columns.tolist()
print(f"Số cột phân loại: {len(cat_cols)}")
print("Một vài cột:", cat_cols[:10])

# Chia nhóm theo cardinality
card_all = X_full[cat_cols].nunique(dropna=False)
low_medium = [c for c in cat_cols if card_all[c] <= 60]   # One-Hot
high_card  = [c for c in cat_cols if card_all[c] >  60]   # Frequency

print("\nNhóm One-Hot (≤60):", low_medium)
print("Nhóm High-card (Frequency) (>60):", high_card)

# Các bộ ánh xạ/mô hình hoá
RARE_THRESHOLD = 0.005  # 0.5%

rare_maps = {}          
ohe = None              
ohe_cols = []           

freq_maps = {}          # {col: pd.Series(label -> freq)} cho nhóm high-card
priors   = {}           # {col: float} (prior khi gặp nhãn unseen)

def build_rare_maps(X: pd.DataFrame, cols: list, thr: float = RARE_THRESHOLD):
    maps = {}
    for c in cols:
        freq = X[c].astype(str).value_counts(normalize=True, dropna=False)
        rare_labels = set(freq[freq < thr].index.astype(str))
        maps[c] = rare_labels
    return maps

def apply_group_rare(s: pd.Series, col: str) -> pd.Series:
    x = s.astype(str)
    x = x.where(~x.isin(rare_maps[col]), "__OTHER__")
    x = x.fillna("__MISSING__").replace({"nan": "__MISSING__"})
    return x

def build_freq_maps(X: pd.DataFrame, cols: list):
    fmap, prior_map = {}, {}
    for c in cols:
        ser = X[c].astype(str)
        freq = ser.value_counts(normalize=True, dropna=False)
        prior = freq.mean()  # prior mềm cho nhãn unseen
        fmap[c] = freq
        prior_map[c] = prior
    return fmap, prior_map

def freq_encode_series(s: pd.Series, col: str) -> pd.Series:
    x = s.astype(str)
    mapped = x.map(freq_maps[col])
    return mapped.fillna(priors[col])

# FIT các encoder/mapping trên TOÀN BỘ dữ liệu hiện có
# 3.1 Rare maps cho OHE
rare_maps = build_rare_maps(X_full, low_medium, thr=RARE_THRESHOLD)

if low_medium:
    X_ohe_base = pd.DataFrame(index=X_full.index)
    for c in low_medium:
        X_ohe_base[c] = apply_group_rare(X_full[c], c)
    ohe = OneHotEncoder(
        sparse_output=False,     # nếu muốn tiết kiệm RAM: True
        handle_unknown="ignore",
        drop="first"             # chống đa cộng tuyến
    )
    ohe_arr = ohe.fit_transform(X_ohe_base.astype(str))
    ohe_cols = ohe.get_feature_names_out(low_medium).tolist()
    df_ohe = pd.DataFrame(ohe_arr, columns=ohe_cols, index=X_full.index)
else:
    df_ohe = pd.DataFrame(index=X_full.index)
    ohe_cols = []

freq_maps, priors = build_freq_maps(X_full, high_card)
df_freq = pd.DataFrame(index=X_full.index)
for c in high_card:
    df_freq[f"{c}__freq"] = freq_encode_series(X_full[c], c)

base_cols = [c for c in X_full.columns if c not in cat_cols]
df_encoded_full = pd.concat(
    [X_full[base_cols].reset_index(drop=True),
     df_ohe.reset_index(drop=True),
     df_freq.reset_index(drop=True)],
    axis=1
)
df_encoded_full.index = X_full.index

print("\nKích thước trước mã hoá:", X_full.shape)
print("Kích thước sau mã hoá :", df_encoded_full.shape)

print("\nCác kiểu dữ liệu sau mã hoá:")
display(df_encoded_full.dtypes.value_counts())
display(df_encoded_full.head(3))


Số cột phân loại: 11
Một vài cột: ['merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob']

Nhóm One-Hot (≤60): ['category', 'gender', 'state']
Nhóm High-card (Frequency) (>60): ['merchant', 'first', 'last', 'street', 'city', 'job', 'dob', 'trans_num']

Kích thước trước mã hoá: (555719, 21)
Kích thước sau mã hoá : (555719, 76)

Các kiểu dữ liệu sau mã hoá:


float64           71
int64              4
datetime64[ns]     1
Name: count, dtype: int64

,trans_date_trans_time,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,...,state_WY,state___OTHER__,merchant__freq,first__freq,last__freq,street__freq,city__freq,job__freq,dob__freq,trans_num__freq
0,2020-06-21 12:14:25,2291163933867244,2.86,29209,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714,...,0.0,0.0,0.001324,0.001152,0.002323,0.001152,0.001152,0.004373,0.001152,0.000002
1,2020-06-21 12:14:33,3573030041201292,29.84,84002,40.3207,-110.4360,302,1371816873,39.450498,-109.960431,...,0.0,0.0,0.001413,0.002715,0.018095,0.001506,0.001506,0.004562,0.001506,0.000002
2,2020-06-21 12:14:53,3598215285024754,41.28,11710,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111,...,0.0,0.0,0.001359,0.010266,0.003880,0.001931,0.001931,0.004655,0.001931,0.000002


## 2d — Lựa chọn đặc trưng (Feature Selection)

**Mục tiêu:**  
Giảm số chiều và loại bỏ thông tin dư thừa/không hữu ích để:
- Tăng **tốc độ huấn luyện** và **độ ổn định** của mô hình,
- Giảm **đa cộng tuyến** và **quá khớp**,
- Cải thiện **khả năng tổng quát hoá**.

### 2c.1. Variance Threshold (loại cột hằng/biến thiên rất thấp)
**Ý tưởng:** Loại đặc trưng có **phương sai ≈ 0** vì không mang thông tin phân biệt.  
**Các bước:**
1. Tính phương sai $ \mathrm{Var}(X_j) $ cho từng cột số sau khi mã hoá (và trước khi scale nếu cần).  
2. Loại cột nếu $ \mathrm{Var}(X_j) \le \tau $ (mặc định $ \tau=0 $ để loại cột hằng; có thể tăng nhẹ để bỏ cột gần-hằng).  

### 2c.2. Loại tương quan cao (Correlation Pruning)
**Ý tưởng:** Khi hai đặc trưng tuyến tính **rất tương quan**, giữ 1 cột và bỏ bớt cột còn lại để giảm trùng lặp.  
**Quy tắc:**  
- Tính ma trận tương quan Pearson giữa các cột số (sau VT).  
- Xét tam giác trên của ma trận, nếu $|\rho_{ij}| > 0.95$ ⇒ đưa một trong hai cột vào danh sách loại.  
- **Tiêu chí giữ lại** có thể là: cột ít thiếu/outlier hơn, dễ diễn giải hơn, hoặc cho kết quả CV tốt hơn.

In [7]:
# Xử lý DateTime

def engineer_datetime_features(df: pd.DataFrame,
                               dt_cols: list,
                               add_cyclic: bool = True,
                               weekend_days: set = {5, 6}) -> pd.DataFrame:
    """
    Biến các cột datetime thành đặc trưng số: year, month, day, dayofweek, hour, is_weekend,
    và (tuỳ chọn) mã hoá chu kỳ sin/cos cho month/dayofweek/hour. Xoá cột gốc datetime.
    """
    out = df.copy()
    for c in dt_cols:
        s = pd.to_datetime(out[c], errors="coerce")
        # Works on pandas 1.x and 2.x
        if getattr(s.dtype, "tz", None) is not None:
            s = s.dt.tz_convert(None)


        # Đặc trưng lịch cơ bản
        year      = s.dt.year.astype("float64")
        month     = s.dt.month.astype("float64")
        day       = s.dt.day.astype("float64")
        dayofweek = s.dt.dayofweek.astype("float64")  # 0=Mon..6=Sun
        hour      = s.dt.hour.astype("float64")
        is_weekend = s.dt.dayofweek.isin(weekend_days).astype("uint8")

        feats = {
            f"{c}_year": year,
            f"{c}_month": month,
            f"{c}_day": day,
            f"{c}_dayofweek": dayofweek,
            f"{c}_hour": hour,
            f"{c}_is_weekend": is_weekend,
        }

        # Mã hoá chu kỳ giúp mô hình hiểu tính tuần hoàn
        if add_cyclic:
            feats[f"{c}_month_sin"] = np.sin(2*np.pi*(month/12.0))
            feats[f"{c}_month_cos"] = np.cos(2*np.pi*(month/12.0))
            feats[f"{c}_dow_sin"]   = np.sin(2*np.pi*(dayofweek/7.0))
            feats[f"{c}_dow_cos"]   = np.cos(2*np.pi*(dayofweek/7.0))
            feats[f"{c}_hour_sin"]  = np.sin(2*np.pi*(hour/24.0))
            feats[f"{c}_hour_cos"]  = np.cos(2*np.pi*(hour/24.0))

        # Gắn vào DataFrame + xử lý thiếu bằng median cho các cột float
        for k, v in feats.items():
            out[k] = v
            if out[k].isna().any():
                out[k] = out[k].fillna(out[k].median())

        # Xoá cột datetime gốc
        out.drop(columns=[c], inplace=True)

    return out

In [8]:


# 0) Cấu hình
TARGET = "is_fraud"
CORR_THRESHOLD = 0.95
APPLY_CORR_PRUNING = True

# 1) Kiểm tra đầu vào tối thiểu
if not isinstance(df, pd.DataFrame):
    raise ValueError("Biến 'df' phải tồn tại và là pandas.DataFrame.")
if TARGET not in df.columns:
    raise ValueError(f"Thiếu cột nhãn {TARGET!r} trong df (dữ liệu gốc).")
if not isinstance(df_encoded_full, pd.DataFrame):
    raise ValueError("Biến 'df_encoded_full' phải tồn tại và là DataFrame đặc trưng đã mã hoá.")

# 2) Lấy nhãn y từ df và căn chỉnh index với df_encoded_full
#    (df_encoded_full thường chỉ chứa FEATURES, không có TARGET)
y = df[TARGET].astype(int).reindex(df_encoded_full.index)
if y.isna().sum() > 0:
    # Nếu index không khớp, dùng thứ tự dòng hiện tại như "fallback"
    # (chỉ dùng khi chắc chắn df và df_encoded_full cùng thứ tự!)
    y = df[TARGET].astype(int).reset_index(drop=True)
    df_encoded_full = df_encoded_full.reset_index(drop=True)

# 3) Sao chép features
X = df_encoded_full.copy()

# 4) Định nghĩa engineer_datetime_features nếu chưa có
def engineer_datetime_features(_df: pd.DataFrame, dt_cols, add_cyclic=True):
    out = _df.copy()
    for c in dt_cols:
        s = pd.to_datetime(out[c], errors="coerce")
        out[f"{c}__year"]  = s.dt.year
        out[f"{c}__month"] = s.dt.month
        out[f"{c}__day"]   = s.dt.day
        out[f"{c}__dow"]   = s.dt.dayofweek
        out[f"{c}__hour"]  = s.dt.hour
        if add_cyclic:
            # Encode chu kỳ cho month/hour (minh hoạ)
            out[f"{c}__month_sin"] = np.sin(2*np.pi*(out[f"{c}__month"].fillna(0))/12)
            out[f"{c}__month_cos"] = np.cos(2*np.pi*(out[f"{c}__month"].fillna(0))/12)
            out[f"{c}__hour_sin"]  = np.sin(2*np.pi*(out[f"{c}__hour"].fillna(0))/24)
            out[f"{c}__hour_cos"]  = np.cos(2*np.pi*(out[f"{c}__hour"].fillna(0))/24)
        # Bỏ cột gốc dạng datetime để tránh lỗi dtype về sau
        out = out.drop(columns=[c])
    return out

# 5) Nếu còn cột datetime, tách đặc trưng thời gian rồi bỏ cột gốc
dt_cols = X.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns.tolist()
if dt_cols:
    X = engineer_datetime_features(X, dt_cols, add_cyclic=True)

# 6) Làm sạch dtype & NaN trước khi chọn đặc trưng
# bool -> uint8
bool_cols = X.select_dtypes(include=["bool"]).columns.tolist()
if bool_cols:
    X[bool_cols] = X[bool_cols].astype(np.uint8)

# điền NaN còn sót
X = X.fillna(0.0)

# ép toàn bộ về float64 (ổn định cho VT & corr)
X_np = X.to_numpy(dtype=np.float64)
print("Số đặc trưng ban đầu:", X.shape[1])

# 7) VarianceThreshold: loại cột hằng (var = 0)
vt = VarianceThreshold(threshold=0.0)
X_vt = vt.fit_transform(X_np)
mask_vt = vt.get_support()
cols_vt = X.columns[mask_vt]

print("Số đặc trưng bị loại bởi VarianceThreshold:", X.shape[1] - len(cols_vt))

# khôi phục DataFrame sau VT
X_vt_df = pd.DataFrame(X_vt, columns=cols_vt, index=X.index)

# 8) Corr-Pruning: loại tương quan rất cao (|ρ| > CORR_THRESHOLD)
if APPLY_CORR_PRUNING and X_vt_df.shape[1] > 1:
    corr = X_vt_df.corr(method="pearson").abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop_corr = {col for col in upper.columns if upper[col].max() > CORR_THRESHOLD}
    selected_cols = [c for c in cols_vt if c not in to_drop_corr]
    X_final = X_vt_df[selected_cols]
    print(f"Số đặc trưng bị loại do tương quan cao (>|ρ|={CORR_THRESHOLD}):", len(to_drop_corr))
else:
    X_final = X_vt_df
    selected_cols = X_final.columns.tolist()

print("Số đặc trưng còn lại:", X_final.shape[1])

# 9) Ghép nhãn tạo bộ tiền xử lý cuối
df_preprocessed = pd.concat([X_final, y], axis=1)
print("Kích thước df_preprocessed:", df_preprocessed.shape)

# Xem nhanh
display(df_preprocessed.head(3))


Số đặc trưng ban đầu: 84
Số đặc trưng bị loại bởi VarianceThreshold: 2
Số đặc trưng bị loại do tương quan cao (>|ρ|=0.95): 4
Số đặc trưng còn lại: 78
Kích thước df_preprocessed: (555719, 79)


,cc_num,amt,zip,lat,long,city_pop,unix_time,category_food_dining,category_gas_transport,category_grocery_net,...,city__freq,job__freq,dob__freq,trans_date_trans_time__day,trans_date_trans_time__dow,trans_date_trans_time__hour,trans_date_trans_time__month_sin,trans_date_trans_time__hour_sin,trans_date_trans_time__hour_cos,is_fraud
0,2.291164e+15,2.86,29209.0,33.9659,-80.9355,333497.0,1.371817e+09,0.0,0.0,0.0,...,0.001152,0.004373,0.001152,21.0,6.0,12.0,1.224647e-16,1.224647e-16,-1.0,0
1,3.573030e+15,29.84,84002.0,40.3207,-110.4360,302.0,1.371817e+09,0.0,0.0,0.0,...,0.001506,0.004562,0.001506,21.0,6.0,12.0,1.224647e-16,1.224647e-16,-1.0,0
2,3.598215e+15,41.28,11710.0,40.6729,-73.5365,34496.0,1.371817e+09,0.0,0.0,0.0,...,0.001931,0.004655,0.001931,21.0,6.0,12.0,1.224647e-16,1.224647e-16,-1.0,0


In [ ]:
# Kiểm tra không còn object và không còn NaN
print("Kiểu dữ liệu sau cùng:")
display(df_preprocessed.dtypes.value_counts())

missing_total = int(df_preprocessed.isna().sum().sum())
print("Tổng số ô thiếu trong df_preprocessed:", missing_total)


Kiểu dữ liệu sau cùng:


float64    78
int64       1
Name: count, dtype: int64

Tổng số ô thiếu trong df_preprocessed: 0
Đã lưu: ../data/tabular/df_preprocessed.csv
